# Analyzing SASUMO outputs

### Import the Required Libraries

In [1]:
import os
import sys
import pathlib

In [2]:
def find_root(path):
    if os.path.split(path)[-1] != "airport-harper-sumo":
        return find_root(os.path.split(path)[0])
    return path


In [3]:
import pathlib

PATH = find_root(pathlib.Path().absolute())
if PATH not in sys.path:
    print(f"adding {PATH} to path")
    sys.path.append(PATH)


adding /home/max/Development/airport-harper-sumo to path


In [4]:
import json5 as json
import sys

import numpy as np
import pandas as pd
import pint
import plotly.graph_objects as go

from SALib.analyze import sobol
# from SASUMO.params import SASUMOConf, ReplayProcessConf


## Defining the Analysis Root

In [5]:
ROOT = r"/home/max/tmp/sasumo-output/CFPaperParametersCorr/07.26.2022_19.21.48"
TOP_LEVEL_SETTINGS = os.path.join(ROOT, "sasumo_params.yaml")
OUTPUT = os.path.join(ROOT, "output.txt")
PROBLEM = os.path.join(ROOT, "SALib_Problem.json")


In [6]:
# with open(TOP_LEVEL_SETTINGS, 'rb') as f:
# settings = Config(os.path.join(ROOT, "sasumo_params.yaml"))

## Open up the Problem

## Creating a table of the Results and Parameters

In [7]:
def gen_import_func():

    from SASUMO.functions.free_flow_speed import FreeFlowSpeed
    from SASUMO.functions.simulation_metrics import SimulationMetrics
    from SASUMO.functions.per_phase_delay import E3SimulationMetrics
    from SASUMO.params import ParameterSweepConf
    from tools.output_tools import USDOTCalibration

    def import_data(
        _dir_name,
        _dir,
    ):
        try:
            with open(os.path.join(_dir, "f_out.txt"), "r") as f:
                d = {"f_out": float(f.read()), "sample_num": int(_dir_name)}
        except Exception as e:
            raise f"issue with {_dir}" from e

        d |= {
            # load the variable values
            **ParameterSweepConf.var_2_records(os.path.join(_dir, "params.yaml")),
            # load the calibration data
            **USDOTCalibration.load_calibration_data(
                os.path.join(_dir, "calibration_results.json"), load_all=True
            ),
            # load the simulation metrics
            **SimulationMetrics.load_traffic_metrics(
                os.path.join(_dir, "simulation_metrics.json")
            ),
            # load the simulation speed
            **FreeFlowSpeed.speed_analysis_loader(
                os.path.join(_dir, "speed_analysis.json")
            ),
            # load the per-phase
            **E3SimulationMetrics.load_traffic_metrics(
                os.path.join(_dir, "per_phase_delay.json")
            ),
        }

        return d

    return import_data

In [8]:
def prepare_df(df: pd.DataFrame) -> pd.DataFrame:

    from tools.output_tools import FuelProperties

    df["Fuel_Gal"] = FuelProperties.joule_2_gal_gas(
        df["f_out"].values * 1e6
    )  # this value is in MJ

    for key in CAL_KEYS:
        if "63082004-EB" not in key:
            res = 0.95 - df[key] if "1.96" in key else 0.666 - df[key]
            res[res < 0] = 0
            df[f"{key}_score"] = 1 - res
        else:
            df[f"{key}_score"] = 1

    df["cal_score"] = df[[f"{key}_score" for key in CAL_KEYS]].mean(axis=1)
    df["Fuel_L"] = df["Fuel_Gal"] * 3.78541178
    df["cal_score_2"] = df[CAL_KEYS_EXTENDED].sum(axis=1)
    # df.drop([f"{key}_score" for key in CAL_KEYS], inplace=True, axis=1)
    return df


In [18]:
FORCE_RECOMPUTE = False
CAL_KEYS = [
    "_".join([key, movement, sigma])
    for key in ["63082002", "63082003", "63082004"]
    for movement in ["WB", "EB"]
    for sigma in ["sigma_1", "sigma_1.96"]
]

CAL_KEYS_EXTENDED = [
    "_".join([key, movement, sigma])
    for key in ["63082002", "63082003", "63082004"]
    for movement in ["WB", "EB", "4"] + (["8"] if key != "63082002" else [])
    for sigma in ["sigma_1", "sigma_1.96"]
]

if FORCE_RECOMPUTE or not os.path.exists(os.path.join(ROOT, "processed_results.csv")):
    print("Processing the SA runs")

    from joblib import Parallel, delayed

    results = Parallel(n_jobs=os.cpu_count() - 1)(
        delayed(gen_import_func())(
            dir.name,
            dir.path,
        )
        for dir in os.scandir(ROOT)
        if dir.is_dir()
    )
    results_df = pd.DataFrame.from_records(results, index="sample_num").sort_index()
    results_df = prepare_df(results_df)
    print(f"Saving processed csv to {os.path.join(ROOT, 'processed_results.csv')}")
    results_df.to_csv(os.path.join(ROOT, "processed_results.csv"))

else:
    print("reading from csv")
    results_df = pd.read_csv(os.path.join(ROOT, "processed_results.csv"))


reading from csv


In [19]:
results_df.tail()

,sample_num,f_out,RandomSeed,63082002_4_sigma_1.96,63082002_4_sigma_1,63082002_4_BDAE,63082002_4_BDSE,63082002_5_sigma_1.96,63082002_5_sigma_1,63082002_5_BDAE,...,63082003_WB_sigma_1.96_score,63082003_EB_sigma_1_score,63082003_EB_sigma_1.96_score,63082004_WB_sigma_1_score,63082004_WB_sigma_1.96_score,63082004_EB_sigma_1_score,63082004_EB_sigma_1.96_score,cal_score,Fuel_L,cal_score_2
95,95,10202.561787,959.636364,1.0,1.0,True,True,0.941176,0.882353,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,316.822132,20.647059
96,96,10409.560952,969.727273,1.0,1.0,True,True,0.882353,0.764706,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,323.250117,20.529412
97,97,10187.957036,979.818182,1.0,1.0,True,True,0.941176,0.823529,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,316.368608,20.647059
98,98,10216.827811,989.909091,1.0,1.0,True,True,1.000000,0.882353,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,317.265138,20.588235
99,99,10209.991839,1000.000000,1.0,1.0,True,True,1.000000,0.882353,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,317.052859,20.647059


In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df.index, y=results_df['cal_score'], mode='lines'))
fig.update_layout(yaxis=dict(title="Calibration"),
                  xaxis=dict(title='Sample Number')
                 )
fig.show()

In [21]:
results_df.sort_values('cal_score_2',).head()

,sample_num,f_out,RandomSeed,63082002_4_sigma_1.96,63082002_4_sigma_1,63082002_4_BDAE,63082002_4_BDSE,63082002_5_sigma_1.96,63082002_5_sigma_1,63082002_5_BDAE,...,63082003_WB_sigma_1.96_score,63082003_EB_sigma_1_score,63082003_EB_sigma_1.96_score,63082004_WB_sigma_1_score,63082004_WB_sigma_1.96_score,63082004_EB_sigma_1_score,63082004_EB_sigma_1.96_score,cal_score,Fuel_L,cal_score_2
22,22,10180.030886,223.000000,1.0,1.0,True,True,1.000000,0.882353,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,316.122476,20.529412
86,86,10222.175223,868.818182,1.0,1.0,True,True,1.000000,0.941176,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,317.431192,20.529412
59,59,10207.594408,596.363636,1.0,1.0,True,True,1.000000,0.823529,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,316.978411,20.529412
94,94,10201.826777,949.545455,1.0,1.0,True,True,1.000000,0.823529,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,316.799308,20.529412
50,50,10236.579239,505.545455,1.0,1.0,True,True,0.941176,0.823529,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,317.878483,20.529412


## Display the top Fuel Consumers

In [13]:
import itertools

MAINLINE_PHASES = ["1", "2", "5", "6"]
SIDE_PHASES = ["3", "4", "7", "8"]

TL = ["63082002", "63082003", "63082004"]

MAINLINE_SUMBOIS = [
    f"{tl}_{p}_meanTimeLoss"
    for tl, p in itertools.product(TL, MAINLINE_PHASES)
    if tl != "63082002" and p not in ["1", "3", "7", "8"]
]
SIDESTREET_SUMBOIS = [
    f"{tl}_{p}_meanTimeLoss"
    for tl, p in itertools.product(TL, SIDE_PHASES)
    if tl != "63082002" and p not in ["1", "3", "7", "8"]
]


In [14]:
results_df["mainline_timeloss"] = results_df[MAINLINE_SUMBOIS].sum(axis=1)
results_df["sidestreet_timeloss"] = results_df[SIDESTREET_SUMBOIS].sum(axis=1)
results_df["timeloss_ratio"] = results_df["sidestreet_timeloss"] / (
    results_df["mainline_timeloss"] + results_df["sidestreet_timeloss"]
)


In [15]:
res = {}
for val in ["Fuel_L", "all_delay_average", "all_travel_time_average", "timeloss_ratio"]:
    res[val] = {
        'q5': results_df[val].quantile(0.05),
        'q95': results_df[val].quantile(0.95)
    }

In [17]:
res

{'Fuel_L': {'q5': 313.18594496708704, 'q95': 320.3336839263023},
 'all_delay_average': {'q5': 39.18076873553245, 'q95': 44.528955706015665},
 'all_travel_time_average': {'q5': 91.32667384941915,
  'q95': 96.68207290611696},
 'timeloss_ratio': {'q5': 0.4073369243279511, 'q95': 0.5075664787085372}}

## Plotting with Plotly

In [22]:
fig = go.Figure()
# for r in results_dfs:
fig.add_trace(go.Scatter(x=results_df.RandomSeed, y=results_df['all_delay_average'], mode='markers'))
# fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Fuel [gal]'], mode='markers', name="Samples", ))
# fig.add_trace(go.Scatter(x=results_poisson_df.index, y=results_poisson_df['Fuel [gal]'], mode='markers', name="Samples", ))
# fig.add_trace(go.Scatter(x=grouped.index, y=grouped['Fuel [gal]'], mode='lines+markers', name="Sample Average"))
# fig.add_trace(go.Scatter(x=grouped.index, y=grouped['Fuel [gal]'] + grouped_std['Fuel [gal]'], mode='lines+markers', name="+1 std dev"))
# fig.add_trace(go.Scatter(x=grouped.index, y=grouped['Fuel [gal]'] - grouped_std['Fuel [gal]'], mode='lines+markers', name="-1 std dev"))
fig.update_layout(yaxis=dict(title="Fuel [gal]"),
                  xaxis=dict(title='Random Seed')
                 )
fig.show()

In [23]:
fig = go.Figure()
# for r in results_dfs:
fig.add_trace(go.Scatter(x=results_df['Fuel_Gal'], y=results_df.all_total_vehicles, mode='markers'))
# fig.add_trace(go.Scatter(x=results_df.index, y=results_df['Fuel [gal]'], mode='markers', name="Samples", ))
# fig.add_trace(go.Scatter(x=results_poisson_df.index, y=results_poisson_df['Fuel [gal]'], mode='markers', name="Samples", ))
# fig.add_trace(go.Scatter(x=grouped.index, y=grouped['Fuel [gal]'], mode='lines+markers', name="Sample Average"))
# fig.add_trace(go.Scatter(x=grouped.index, y=grouped['Fuel [gal]'] + grouped_std['Fuel [gal]'], mode='lines+markers', name="+1 std dev"))
# fig.add_trace(go.Scatter(x=grouped.index, y=grouped['Fuel [gal]'] - grouped_std['Fuel [gal]'], mode='lines+markers', name="-1 std dev"))
fig.update_layout(yaxis=dict(title="Fuel [gal]"),
                  xaxis=dict(title='Random Seed')
                 )
fig.show()

## Plotting with Plotly

In [24]:
import plotly.figure_factory as ff


fig = ff.create_distplot([results_df["all_delay_average"].values], group_labels=["Fuel"], bin_size=.5)
fig.show()

In [25]:
results_df.sort_values('RandomSeed',).head()

,sample_num,f_out,RandomSeed,63082002_4_sigma_1.96,63082002_4_sigma_1,63082002_4_BDAE,63082002_4_BDSE,63082002_5_sigma_1.96,63082002_5_sigma_1,63082002_5_BDAE,...,63082003_WB_sigma_1.96_score,63082003_EB_sigma_1_score,63082003_EB_sigma_1.96_score,63082004_WB_sigma_1_score,63082004_WB_sigma_1.96_score,63082004_EB_sigma_1_score,63082004_EB_sigma_1.96_score,cal_score,Fuel_L,cal_score_2
0,0,10244.830563,1.000000,1.0,1.0,True,True,0.882353,0.823529,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,318.134713,20.588235
1,1,10249.146507,11.090909,1.0,1.0,True,True,0.882353,0.764706,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,318.268736,20.647059
2,2,10260.490206,21.181818,1.0,1.0,True,True,1.000000,0.823529,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,318.620995,20.588235
3,3,10206.654159,31.272727,1.0,1.0,True,True,0.882353,0.882353,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,316.949213,20.529412
4,4,10224.304958,41.363636,1.0,1.0,True,True,1.000000,0.823529,True,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,317.497327,20.529412
